In [1]:
import torch
from diffusers import TextToVideoZeroPipeline
import numpy as np
import cv2

In [2]:
FPS = 10
SECONDS = 60

In [3]:
model_id = "stable-diffusion-v1-5/stable-diffusion-v1-5"
pipe = TextToVideoZeroPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

Couldn't connect to the Hub: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/stable-diffusion-v1-5/stable-diffusion-v1-5 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000020A9D132930>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))"), '(Request ID: e210f360-fbab-4c9d-986c-fe4a90b10aee)').
Will try to load from local cache.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
seed = 0
#video_length = 24  #24 ÷ 4fps = 6 seconds
video_length = FPS * SECONDS
chunk_size = 8

In [5]:
prompt = '''
Grand Theft Auto Six Trailer in 3D with realistic graphics showing various parts of Vice City map.
'''

In [6]:
# Generate the video chunk-by-chunk
result = []
chunk_ids = np.arange(0, video_length, chunk_size - 1)
generator = torch.Generator(device="cuda")
for i in range(len(chunk_ids)):
    print(f"Processing chunk {i + 1} / {len(chunk_ids)}")
    ch_start = chunk_ids[i]
    ch_end = video_length if i == len(chunk_ids) - 1 else chunk_ids[i + 1]
    # Attach the first frame for Cross Frame Attention
    frame_ids = [0] + list(range(ch_start, ch_end))
    # Fix the seed for the temporal consistency
    generator.manual_seed(seed)
    output = pipe(prompt=prompt, video_length=len(frame_ids), generator=generator, frame_ids=frame_ids)
    result.append(output.images[1:])

# Concatenate chunks and save
result = np.concatenate(result)
result = [(r * 255).astype("uint8") for r in result]

Processing chunk 1 / 86


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

e:\Softwares\Programming Languages\Python\Python312\Lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
e:\Softwares\Programming Languages\Python\Python312\Lib\site-packages\torch\nn\functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


  0%|          | 0/48 [00:00<?, ?it/s]

...

  0%|          | 0/48 [00:00<?, ?it/s]

Processing chunk 86 / 86


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

In [7]:
def save_video_from_frames(frames, output_filename, fps=24):
    """
    Save a list of frames as an MP4 video.

    Parameters:
    - frames: List of frames (numpy arrays of shape (height, width, 3)).
    - output_filename: Output file name (e.g., 'output.mp4').
    - fps: Frames per second for the video.
    """
    if not frames:
        print("No frames to save.")
        return

    # Get the size of the frames
    height, width, _ = frames[0].shape

    # Define the codec and create the VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_filename, fourcc, fps, (width, height))

    for frame in frames:
        video_writer.write(frame)

    video_writer.release()
    print(f"Video saved as {output_filename}")

In [8]:
# import imageio
# imageio.mimsave("Output.mp4", result, format = 'mp4')

In [9]:
# Save the video
save_video_from_frames(result, 'Output.mp4', fps=FPS)

Video saved as Output.mp4
